#### Cross-Encoder Passage Retrieval Model

We will now explore an alternative mnodel for document retreival, called a `cross-encoder`. Unlike the bi-encoder, we will use a single BERT model and feed it input sequence which is a concatenation of a claim-passage pair. Then using the output embedding of the [CLS] token, we perform a `binary classification` of whether or not, the passage is relevant to this claim or not. We can set up training instances of both positive/relevant and negative/non-relevant pairs and train using binary cross-entropy loss. Then the sigmoid of the output logit can be interpreted as a relevancy score between $[0,1]$. 

Since each claim can have multiple relevant evidence passages, we can create multiple positive pairs. Then to have a balanced distribution of the two classes, we would also create the same number of negative pairs. However, we need to figure out a way to select the negative passages. The simplest way is to just `randomly sample` a passage from the document store which is also not in the list of positive passages. Howerever, these random samples may be too easy for our model to detect, and so ideally we would want to use some form of `hard-negative mining` to select good negative passages. 

So first, lets set up our dataset and implement hard-negative mining. 